# RAG & Prompt

In [17]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["HF_TOKNE"] = userdata.get('HF_TOKEN')

## 데이터 로드 및 청크

In [26]:
import json
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

json_filename = 'ajou_2023_1.json'
json_dir = './data'
json_file_path = os.path.join(json_dir, json_filename)
# json 파일 로드
with open('ajou_2023_1.json', 'r', encoding='utf-8') as f:
  data = json.load(f)

print('json 필드:', data.keys())
print('question_id:', data['question_id'])

# Text Splitter 초기화 (청크 작업)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# metadata 확인 및 추가
initial_docs = []
base_metadata = {
    "university": "아주대학교",
    "year": 2023,
    "subject": "인문논술"
}

# content_type과 해당 내용을 매핑
content_map = {
    "출제의도": data.get("intended_purpose"),
    "채점기준": data.get("grading_criteria"),
    "모범답안": data.get("sample_answer")
}

for content_type, content in content_map.items():
    if content:
        # content를 청크로 분할, 각 청크는 원본 doc의 메타데이터를 상속
        metadata={
            **base_metadata,
            "content_type": content_type,
            "source_file": json_filename
        }
        # chunk
        chunks = text_splitter.create_documents(
            texts=[content],
            metadatas=[metadata]
        )
        docs.append(chunks)

print(len(docs))
docs[0]

json 필드: dict_keys(['question_id', 'intended_purpose', 'grading_criteria', 'sample_answer'])
question_id: 2023_아주대_인문_1
3


Document(metadata={'university': '아주대학교', 'year': 2023, 'subject': '인문논술', 'content_type': '출제의도', 'question_id': '2023_아주대_인문_1'}, page_content='\n')

## 임베딩 및 벡터db 저장

In [ ]:
# 문서 임베딩 및 벡터db 저장
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import torch

# embedding = OpenAIEmbeddings 사용
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# vector_db = FAISS
vector_db = FAISS.from_documents(
    documents=docs,
    embedding=embeddings
)
vector_db.save_local('./db/faiss')

# test

In [ ]:
pip install langchain langchain-openai faiss-cpu langchain_core python-dotenv

## 파일 경로 설정 및 모델 설정
- 파일 경로(json, pdf, faiss_db)
- Text Splitter
- Embedding

In [ ]:
import json
import os

from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# --- 1. 설정 변수 정의 ---
JSON_DATA_DIR = './data_json'  # 논술 문제 JSON 파일들이 저장된 디렉토리 경로
PDF_FILES_DIR = './test_pdf' # 논술 문제 PDF 파일들이 저장된 디렉토리 (벡터 DB에 직접 포함X, 메타데이터로만 연결)
BASE_FAISS_DB_DIR = './faiss_test_db' # 각 문제별 FAISS 인덱스(DB)를 저장할 최상위 디렉토리

# --- 2. 텍스트 분할기(Text Splitter) 초기화 ---
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=['\n\n', '\n', '.', ' ', ''],
    length_function=len,
    is_separator_regex=False,
)

# --- 3. 임베딩 모델 설정 ---
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

## FAISS 문제별 인덱스 생성 및 Document & Chunk 설정
- json 디렉토리를 내 파일을 순회하면서 인덱스를 생성한다 *filename = FAISS 인덱스명
- 각 인덱스마다 Document를 생성한다
- Document -> chunk 한후
- 벡터db에 from_document 방식으로 저장한다

In [ ]:
# --- 4. 모든 JSON 파일을 순회하며 각 문제별 FAISS 인덱스 생성 ---
for filename in os.listdir(JSON_DATA_DIR):
    if filename.endswith('.json'):
        json_file_path = os.path.join(JSON_DATA_DIR, filename) # 현재 JSON 파일의 전체 경로

        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f) # JSON 데이터를 파이썬 딕셔너리로 변환!! json.load()

        # metadata 추가하기
        question_id = data["question_id"] # 무조건 존재 -> FAISS 인덱스명
        university = data.get("university", "Unknown University")
        year = data.get("year", 0)
        subject = data.get("subject", "Unknown Subject")

        # pdf *파일명이 json과 동일할 것
        pdf_filename = f"{os.path.splitext(filename)[0]}.pdf"
        pdf_path = os.path.join(PDF_FILES_DIR, pdf_filename)

        docs = []

        base_metadata = {
            "university": university,
            "year": year,
            "subject": subject,
            "question_id": question_id,
            "json_path": filename, # 이 청크의 원본 JSON 파일명
            "pdf_path": pdf_path, # 이 청크와 관련된 PDF 파일의 경로
        }

        content = {
            "출제의도": data.get("intended_purpose"),
            "채점기준": data.get("grading_criteria"),
            "모범답안": data.get("sample_answer")
        }

        for content_type, content_text in content.items():
            if content_text:
                doc = Document(
                    page_content=content_text,
                    metadata={
                        **base_metadata,
                        "content_type": content_type,
                    }
                )
                docs.append(doc)

        # Document -> chunk
        chunks = splitter.split_documents(docs)


        # 문제별 FAISS 인덱스 생성 및 저장
        if chunks:
            test_faiss_path = os.path.join(BASE_FAISS_DB_DIR, question_id) # (예: './faiss_test_db/{question_id}')
            os.makedirs(test_faiss_path, exist_ok=True) # 없다면 생성

            vector_db = FAISS.from_documents(
                documents=chunks, # Document 후 자른 chunk를 전달!
                embedding=embeddings
            )
            # FAISS DB를 로컬 파일로 저장: index.faiss(벡터데이터)와 index.pkl(메타데이터)
            vector_db.save_local(test_faiss_path)
            print(f" '{question_id}' ({len(chunks)} 청크) 생성 완료: {test_faiss_path}")


print(f"\n모든 문제별 FAISS 인덱스 생성이 완료되었습니다. 저장 경로: '{BASE_FAISS_DB_DIR}'")

## FAISS 인덱스 로드 및 직접 사용
- FAISS.load_local

In [ ]:
selected_question_id = "2024_아주대_인문_1"

selected_faiss_path = os.path.join(BASE_FAISS_DB_DIR, selected_question_id)

if os.path.exists(selected_faiss_path):
    loaded_vector_db = FAISS.load_local(selected_faiss_path, embeddings, allow_dangerous_deserialization=True)
    print(f"'{selected_question_id}' 인덱스가 성공적으로 로드되었습니다.")

    query = "이 문제의 채점 기준에 대해 자세히 알려줘."

    retrieved_docs = vector_db.similarity_search(
        query,
        k=3,
        filter={"content_type": "채점기준"}
    )

    print(f"\n쿼리: '{query}'에 대한 검색 결과:")
    for i, doc in enumerate(retrieved_docs):
        print(f"메타데이터: {doc.metadata}")
        print(f"내용 미리보기: {doc.page_content[:200]}...")

## Retriever 및 Chain 생성

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-4.1", temperature=0)
retriever = vector_db.as_retriever({"k": 3, "filter": {"content_type": "채점기준"} })

prompt = PromptTemplate.from_template(
    "다음은 논술문제 관련 질의입니다. 질문: {query}"
)

selected_question_id = "2024_아주대_인문_1" # {question_id} 의 content
selected_faiss_path = os.path.join(BASE_FAISS_DB_DIR, selected_question_id)
vector_db = FAISS.load_local(
    documents=selected_faiss_path,
    embedding=embeddings,
    allow_dangerous_deserialization=True # allow_dangerous_deserialization
    )


rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)
query = "이 문제의 채점 기준에 대해 자세히 알려줘."
response = rag_chain.invoke({"query": query})

print(f"답변: {response['result']}")

# TEST

## requests

In [7]:
%pip install langchain langchain-openai faiss-cpu langchain_core python-dotenv langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


## 환경설정 import

In [8]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [9]:
import json
from dotenv import load_dotenv

from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

load_dotenv()

False

## 설정 및 변수 선언

In [21]:
JSON_DIR = './data_json'
PDF_DIR = './test_pdf'
VECTOR_DB_DIR = './test_vector_db'

# text splitter 준비
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=['\n\n', '\n', '.', ' ', ''],
    length_function=len,
    is_separator_regex=False,
)

# 임베딩 모델 및 LLM
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.01)

print("설정 변수 정의 및 초기화 완료!")

설정 변수 정의 및 초기화 완료!


## FAISS 인덱스 생성 및 저장

In [22]:
chunks = []

# 디렉토리 생성 (없으면 만들기)
os.makedirs(JSON_DIR, exist_ok=True)
os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(VECTOR_DB_DIR, exist_ok=True)

for filename in os.listdir(JSON_DIR):
    if filename.endswith('.json'):
        json_file_path = os.path.join(JSON_DIR, filename)

        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f) # JSON -> 파이썬(딕셔너리)

        # 메타데이터 생성
        print(os.path.splitext(filename)[0])
        question_id = data.get("question_id", os.path.splitext(filename)[0])
        # university = data.get("university", "Unknown University")
        # year = data.get("year", 0)
        # subject = data.get("subject", "Unknown Subject")

        content_map = {
            "출제의도": data.get("intended_purpose"),
            "채점기준": data.get("grading_criteria"),
            "모범답안": data.get("sample_answer")
        }

        current_docs = []
        for content_type, content_text in content_map.items():
            if content_text:
                doc = Document(
                    page_content=content_text,
                    metadata={
                        # "university": university,
                        # "year": year,
                        # "subject": subject,
                        "question_id": question_id,
                        "content_type": content_type,
                    }
                )
                current_docs.append(doc)

        # 현재 파일의 document(current_docs)를 chunk(current_chunk) 함 -> chunks의 저장
        current_chunk = splitter.split_documents(current_docs)
        chunks.extend(current_chunk) # extend 사용!

ajou_2023_1
ajou_2024_1


In [23]:
# FAISS 인덱스 생성
if chunks:
    vector_db = FAISS.from_documents(
        documents=chunks, # 모든 청크를 document로!!
        embedding=embeddings
    )
    # FAISS DB를 로컬 파일 저장
    vector_db.save_local(VECTOR_DB_DIR)
    print(f"\n모든 문제 데이터({len(chunks)} 청크)를 포함하는 단일 FAISS 인덱스 생성 완료: '{VECTOR_DB_DIR}'")
else:
    print("\n경고: 처리할 JSON 데이터가 없습니다. FAISS 인덱스를 생성하지 않았습니다.")


모든 문제 데이터(0 청크)를 포함하는 단일 FAISS 인덱스 생성 완료: './test_vector_db'


## FAISS 인덱스 로드 및 Retriever 생성

In [24]:
# 저장된 통합 FAISS DB를 메모리로 불러옵니다.
loaded_vector_db = FAISS.load_local(
    VECTOR_DB_DIR,
    embeddings,
    allow_dangerous_deserialization=True
)
print(f"FAISS 인덱스가 성공적으로 로드되었습니다. 경로: '{VECTOR_DB_DIR}'")

# Retriever 생성: 모든 문서에서 검색
retriever = loaded_vector_db.as_retriever(search_kwargs={"k": 5})
print("Retriever 생성 완료!")

FAISS 인덱스가 성공적으로 로드되었습니다. 경로: './test_vector_db'
Retriever 생성 완료!


## Chain 생성
- 논술 답안 첨삭 체인
- 논술 답안 첨삭 관련 질문 챗봇 체인

In [25]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def get_doc_text(query: str) -> str:
    docs = retriever.invoke(query)
    return "\n".join(doc.page_content for doc in docs)


# 1. 프롬프트 템플릿
essay_prompt = PromptTemplate.from_template("""
당신은 대학입시 논술전형 채점관입니다. 다음 정보를 바탕으로 학생의 답안을 엄격하게 평가하고, 상세한 피드백을 제공해주세요.

---
**논술 문제 정보:**
- 출제 의도: {intended_purpose}
- 채점 기준: {grading_criteria}
- 모범 답안: {sample_answer}

---
**학생의 답안:**
{user_essay}

---
**평가 가이드라인:**
(이하 생략...)
""")

# 2. RAG 기반 체인 구성
essay_chain = (
    RunnableParallel(
        {
            # 사용자가 입력한 에세이
            "user_essay": lambda x: x["user_essay"],

            # 검색 기반 필드 - 질문 정보 기반 쿼리
            "intended_purpose": RunnableLambda(lambda x: get_doc_text(f"{x['question_info']} 출제의도")),
            "grading_criteria": RunnableLambda(lambda x: get_doc_text(f"{x['question_info']} 채점기준")),
            "sample_answer": RunnableLambda(lambda x: get_doc_text(f"{x['question_info']} 모범답안")),
        }
    )
    | essay_prompt
    | llm
    | StrOutputParser()
)

print("논술 첨삭 체인 생성 완료!")

논술 첨삭 체인 생성 완료!


In [26]:
# 논술 관련 질의응답 챗봇 체인
essay_qa_prompt = PromptTemplate.from_template("""
주어진 문맥 정보를 바탕으로 질문에 답변해주세요.
만약 문맥 정보에 답변이 없다면, "주어진 정보만으로는 답변하기 어렵습니다."라고 답해주세요.
추가적인 정보는 생성하지 마세요.

문맥:
{context}

질문: {query}
답변:"""
)

# Retriever -> 프롬프트 -> LLM -> 결과 파싱
essay_qa_chain = (
    RunnableParallel(
        {"context": retriever | (lambda docs: "\n\n".join([doc.page_content for doc in docs])),
         "question": RunnablePassthrough()}
    )
    | essay_qa_prompt
    | llm
    | StrOutputParser()
)
print("논술 관련 질의응답 챗봇 체인 생성완료!")

논술 관련 질의응답 챗봇 체인 생성완료!


In [27]:
output = essay_chain.invoke({
    "question_info": "2023 한국외대 인문논술 1번",
    "user_essay": "다른 학부모가 배신하면 자신도 배신한다."
})
print(output)

학생의 답안을 평가하기 위해 주어진 정보를 바탕으로 분석해보겠습니다.

**학생의 답안 분석:**
학생의 답안은 "다른 학부모가 배신하면 자신도 배신한다."라는 한 문장으로 구성되어 있습니다. 이 문장은 주어진 문제의 요구사항을 충분히 반영하지 못하고 있습니다. 문제 2-2는 (다)와 (라)의 학부모의 행동을 비교하고, 그 결과를 예측하는 것을 요구하고 있습니다. 학생의 답안은 (라)의 학부모의 행동을 부분적으로 언급하고 있지만, (다)의 학부모의 행동이나 두 학부모의 행동을 비교하는 내용이 전혀 포함되어 있지 않습니다.

**채점 기준에 따른 평가:**
1. **(다)의 학부모의 행동 예측 (4점):** 학생의 답안에는 (다)의 학부모에 대한 언급이 전혀 없습니다. 따라서 이 부분에 대한 점수는 0점입니다.
   
2. **(라)의 학부모의 행동 예측 (4점):** 학생의 답안은 (라)의 학부모가 "다른 학부모가 배신하면 자신도 배신한다"는 점을 언급하고 있습니다. 이 부분은 부분적으로 맞지만, (라)의 학부모가 협력할 경우에 대한 언급이 없으므로 완전한 답변이 아닙니다. 따라서 이 부분에 대한 점수는 2점입니다.

3. **(다)와 (라)의 학부모의 선호 비교 (3점):** 학생의 답안은 (다)와 (라)의 학부모의 선호를 비교하지 않았습니다. 따라서 이 부분에 대한 점수는 0점입니다.

4. **(다)의 학부모의 선호 설명 (3점):** (다)의 학부모에 대한 설명이 없으므로 이 부분에 대한 점수는 0점입니다.

5. **(라)의 학부모의 선호 설명 (3점):** 학생의 답안은 (라)의 학부모의 배신에 대한 선호를 언급했지만, 협력에 대한 선호는 언급하지 않았습니다. 따라서 이 부분에 대한 점수는 1점입니다.

**총점: 3/17**

**피드백:**
학생의 답안은 문제의 요구사항을 충분히 반영하지 못했습니다. (다)와 (라)의 학부모의 행동을 모두 설명하고, 그들의 행동을 비교하는 것이 필요합니다. 또한, 각 학부모의 행동에 대한 예측과 그 이유를 명확히